In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
# Tools
import sqlite3
import pandas as pd
from datetime import date

def get_today_date():
    """
    Returns today's date as a string in the format YYYY-MM-DD.
    """
    return date.today().isoformat()

def bookTimeEntry(project: str, date, hours: int = 8):
    """
    Book time entry for a project.
    Args:
        projectId (str): The project ID.
        date (str): The date for the time entry.
        hours (int): The number of hours to book.
    """
    return f"Book time entry for {project}."

def get_projects():
    """
    Read the timetracker.db SQLite database and return a list of projects
    with Project Name, Client Name, and Project ID.
    
    Returns:
        DataFrame: A pandas DataFrame containing project information
    """
    # Connect to the SQLite database
    conn = sqlite3.connect('/workspaces/agent-poc/packages/api/timetracker.db')
    
    # Create a SQL query that joins the Projects and Clients tables
    query = """
    SELECT p.Id as ProjectId, p.Name as ProjectName, c.Name as ClientName
    FROM Projects p
    JOIN Clients c ON p.ClientId = c.Id
    ORDER BY c.Name, p.Name
    """
    
    # Execute the query and load results into a DataFrame
    projects_df = pd.read_sql_query(query, conn)
    
    # Close the connection
    conn.close()
    
    return projects_df

tools = [bookTimeEntry, get_projects]

In [7]:
from langchain_anthropic import ChatAnthropic
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display

llm = ChatAnthropic(model="claude-3-7-sonnet-20250219")

llm_with_tools = llm.bind_tools(tools)

# System message
sys_msg = SystemMessage(
    content="You are a terse assistant tasked with helping " \
    "Intertech employees track time spent on projects. " \
    "Today is " + get_today_date() + ".")

def assistant(state: MessagesState):
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

# Graph
builder = StateGraph(MessagesState)

# Nodes
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

# Edges
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")
react_graph = builder.compile()


In [8]:
messages = [HumanMessage(content="I spent all day on Wednesday working on the mobile app development.")]
messages = react_graph.invoke({"messages": messages})

for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

I spent all day on Wednesday working on the mobile app development.
================================== Ai Message ==================================

[{'text': "I'll help you log your time entry for Wednesday. Let me first check the available projects to find the mobile app development project.", 'type': 'text'}, {'id': 'toolu_01HawdvLYR68iDGmHRaWvp2x', 'input': {}, 'name': 'get_projects', 'type': 'tool_use'}]
Tool Calls:
  get_projects (toolu_01HawdvLYR68iDGmHRaWvp2x)
 Call ID: toolu_01HawdvLYR68iDGmHRaWvp2x
  Args:
================================= Tool Message =================================
Name: get_projects

                               ProjectId  \
0   D746AB82-BCA4-4FD1-9CAA-C12CEE5976D9   
1   B06E3093-9C41-4770-B74A-52FD80C4913C   
2   BCC51961-1BC7-4267-9648-5416EC556EDD   
3   7DFBB9D9-58C2-4056-8F0F-C14B8ACD517C   
4   E1D598F0-E6DD-43B1-8E6D-5F7CF903900A   
5   2509D9C7-2B5F-498B-B766-28